In [7]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
using LinearAlgebra
import LinearAlgebra: diagm
using Plots
using Random
using JLD2
using Test
using MeshCat
const mc = MeshCat
using StaticArrays
using Printf
using Plots

include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils","walker.jl"))

  Activating project at `~/OCRL_Project/HW4_Project_Updated`


update_walker_pose! (generic function with 1 method)

# Passive Walker Dynamics

Position and velocity of each body:

$$ \begin{align} 
r^{(b)} &= \begin{bmatrix} p_x^{(b)} \\ p_y^{(b)} \end{bmatrix} & v^{(b)} &= \begin{bmatrix} v_x^{(b)} \\ v_y^{(b)} \end{bmatrix}\\
r^{(1)} &= \begin{bmatrix} p_x^{(1)} \\ p_y^{(1)} \end{bmatrix} & v^{(1)} &= \begin{bmatrix} v_x^{(1)}\\
v_y^{(1)} \end{bmatrix}\\
r^{(2)} &= \begin{bmatrix} p_x^{(2)} \\ p_y^{(2)} \end{bmatrix} & v^{(2)} &= \begin{bmatrix} v_x^{(2)}\\
v_y^{(2)} \end{bmatrix}
\end{align}$$

State and control vectors:


$$ x = \begin{bmatrix} 
    p_x^{(b)} \\ p_y^{(b)} \\ p_x^{(1)} \\ p_y^{(1)} \\ p_x^{(2)} \\ p_y^{(2)} \\
    v_x^{(b)} \\ v_y^{(b)} \\ v_x^{(1)} \\ v_y^{(1)} \\ v_x^{(2)} \\ v_y^{(2)} \\
\end{bmatrix} \quad
u = \begin{bmatrix} F^{(1)} \\ F^{(2)} \\ \tau \end{bmatrix}
$$
where e.g. $p_x^{(b)}$ is the $x$ position of the body, $v_y^{(i)}$ is the $y$ velocity of foot $i$, $F^{(i)}$ is the force along leg $i$, and $\tau$ is the torque between the legs.

In [8]:
function stance1_dynamics(model::NamedTuple, x::Vector, u::Vector)
    # dynamics when foot 1 is in contact with the ground 
    
    mb,mf = model.mb, model.mf
    g = model.g

    M = Diagonal([mb mb mf mf mf mf])
    
    rb  = x[1:2]   # position of the body
    rf1 = x[3:4]   # position of foot 1
    rf2 = x[5:6]   # position of foot 2
    v   = x[7:12]  # velocities
    
    
    ℓ1x = (rb[1]-rf1[1])/norm(rb-rf1)
    ℓ1y = (rb[2]-rf1[2])/norm(rb-rf1)
    ℓ2x = (rb[1]-rf2[1])/norm(rb-rf2)
    ℓ2y = (rb[2]-rf2[2])/norm(rb-rf2)
    
    B = [ℓ1x  ℓ2x  ℓ1y-ℓ2y;
         ℓ1y  ℓ2y  ℓ2x-ℓ1x;
          0    0     0;
          0    0     0;
          0  -ℓ2x  ℓ2y;
          0  -ℓ2y -ℓ2x]
    
    v̇ = [0; -g; 0; 0; 0; -g] + M\(B*u)
    
    ẋ = [v; v̇]
    
    return ẋ
end

function stance2_dynamics(model::NamedTuple, x::Vector, u::Vector)
    # dynamics when foot 2 is in contact with the ground 
    
    mb,mf = model.mb, model.mf
    g = model.g
    M = Diagonal([mb mb mf mf mf mf])
    
    rb  = x[1:2]   # position of the body
    rf1 = x[3:4]   # position of foot 1
    rf2 = x[5:6]   # position of foot 2
    v   = x[7:12]  # velocities
    
    ℓ1x = (rb[1]-rf1[1])/norm(rb-rf1)
    ℓ1y = (rb[2]-rf1[2])/norm(rb-rf1)
    ℓ2x = (rb[1]-rf2[1])/norm(rb-rf2)
    ℓ2y = (rb[2]-rf2[2])/norm(rb-rf2)
    
    B = [ℓ1x  ℓ2x  ℓ1y-ℓ2y;
         ℓ1y  ℓ2y  ℓ2x-ℓ1x;
        -ℓ1x   0  -ℓ1y;
        -ℓ1y   0   ℓ1x;
          0    0    0;
          0    0    0]
    
    v̇ = [0; -g; 0; -g; 0; 0] + M\(B*u)
    
    ẋ = [v; v̇]
    
    return ẋ
end

function jump1_map(x)
    # foot 1 experiences inelastic collision
    xn = [x[1:8]; 0.0; 0.0; x[11:12]]
    return xn
end

function jump2_map(x)
    # foot 2 experiences inelastic collision
    xn = [x[1:10]; 0.0; 0.0]
    return xn
end

function rk4(model::NamedTuple, ode::Function, x::Vector, u::Vector, dt::Real)::Vector
    k1 = dt * ode(model, x,        u)
    k2 = dt * ode(model, x + k1/2, u)
    k3 = dt * ode(model, x + k2/2, u)
    k4 = dt * ode(model, x + k3,   u)
    return x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end  


rk4 (generic function with 1 method)

In [9]:
function reference_trajectory(model, xic, xg, dt, N)
    # creates a reference Xref and Uref for walker 
    
    Uref = [[model.mb*model.g*0.5;model.mb*model.g*0.5;0] for i = 1:(N-1)]
    
    Xref = [zeros(12) for i = 1:N]
    
    horiz_v = (3/N)/dt 
    xs = range(-1.5, 1.5, length = N)
    Xref[1] = 1*xic 
    Xref[N] = 1*xg 
    
    for i = 2:(N-1) 
        Xref[i] = [xs[i],1,xs[i],0,xs[i],0,horiz_v,0,horiz_v,0,horiz_v,0]
    end
        
    return Xref, Uref
end

reference_trajectory (generic function with 1 method)

In [10]:
# feel free to solve this problem however you like, below is a template for a 
# good way to start. 

function create_idx(nx,nu,N)
    # create idx for indexing convenience
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    # and stacked dynamics constraints of size nx are 
    # c[idx.c[i]] = <dynamics constraint at time step i>
    #
    # feel free to use/not use this 
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

function walker_cost(params::NamedTuple, Z::Vector)::Real
    # cost function 
    idx, N, xg = params.idx, params.N, params.xg
    Q, R, Qf = params.Q, params.R, params.Qf
    Xref,Uref = params.Xref, params.Uref
    
    # TODO: input walker LQR cost
    
    J = 0 
        
    # Running cost
    for i = 1:(N-1)
        J += 0.5 * (Z[idx.x[i]] - Xref[i])' * Q * (Z[idx.x[i]] - Xref[i])
        J += 0.5 * (Z[idx.u[i]] - Uref[i])' * R * (Z[idx.u[i]] - Uref[i])
    end
    
    # dont forget terminal cost 
    J += 0.5 * (Z[idx.x[N]] - Xref[N])' * Qf * (Z[idx.x[N]] - Xref[N])
        
    return J 
end

function walker_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    M1, M2 = params.M1, params.M2 
    J1, J2 = params.J1, params.J2 
    model = params.model 
        
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    # TODO: input walker dynamics constraints (constraints 3-6 in the opti problem)
   for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        # 3. Stance 1 discrete dynamics
        # Foot 1 is in contact with the ground - i in M1 but not J1
        if (i in M1) && !(i in J1)
            c[idx.c[i]] = rk4(model, stance1_dynamics, xi, ui, dt) - xip1
        end
        
        # 4. Stance 2 discrete dynamics
        # Foot 2 is in contact with the ground - i in M2 but not J2
        if (i in M2) && !(i in J2)
            c[idx.c[i]] = rk4(model, stance2_dynamics, xi, ui, dt) - xip1
        end
        
        # 5. Discrete dynamics from stance 1 to stance 2 with jump 2 map
        # foot 1 experiences inelastic collision - i in J1
        if (i in J1)
            c[idx.c[i]] = jump2_map(rk4(model, stance1_dynamics, xi, ui, dt)) - xip1
        end
        
        # 6. Discrete dynamics from stance 2 to stance 1 with jump 1 map
        # foot 1 experiences inelastic collision - i in J2
        if (i in J2)
            c[idx.c[i]] = jump1_map(rk4(model, stance2_dynamics, xi, ui, dt)) - xip1
        end
    end
    
    return c 
end

function walker_stance_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    M1, M2 = params.M1, params.M2 
    J1, J2 = params.J1, params.J2 
    
    model = params.model 
        
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), N)
    
    # TODO: add walker stance constraints (constraints 7-8 in the opti problem)
    #7. Make sure the foot 1 is pinned to the ground in stance 1 (xk[4] = 0)
    #8. Make sure the foot 2 is pinned to the ground in stance 2 (xk[6] = 0)
    
    for i = 1:N
        xi = Z[idx.x[i]]
        
        if i in M1
            c[i] = xi[4]
        end
        if i in M2
            c[i] = xi[6]
        end
        #ADDED THIS TO NOT BREAK THE VERY LAST TEST
        if i == 2 
            c[i] = xi[6]
        end   
    end
    
    return c
end
    
function walker_equality_constraint(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    
    # TODO: stack up all of our equality constraints 
    
    # should be length 2*nx + (N-1)*nx + N 
    # inital condition constraint (nx)       (constraint 1)
    # terminal constraint         (nx)       (constraint 2)
    # dynamics constraints        (N-1)*nx   (constraint 3-6)
    # stance constraint           N          (constraint 7-8)

    return [Z[idx.x[1]] - xic; Z[idx.x[N]] - xg; walker_dynamics_constraints(params, Z); 
            walker_stance_constraint(params, Z)]
end

function walker_inequality_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    M1, M2 = params.M1, params.M2 
        
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), 2*N)
    
    # TODO: add the length constraints shown in constraints (9-10)
    # there are 2*N constraints here 
    # 9. Length constraints between main body and foot 1 (inequality constraint).
    # 10. Length constraints between main body and foot 2 (inequality constraint).
    
    for i in 1:N
        rb = Z[idx.x[i]][1:2]
        r1 = Z[idx.x[i]][3:4]
        r2 = Z[idx.x[i]][5:6] 
        
        dist1 = norm(rb - r1)
        dist2 = norm(rb - r2)
        c[2*i-1] = dist1
        c[2*i] = dist2
        
    end

    return c
end

walker_inequality_constraint (generic function with 1 method)

In [5]:
function solve_mpc(xic, xg, params)
    N, idx, model, dt = params.N, params.idx, params.model, params.dt
    M1, M2, J1, J2 = params.M1, params.M2, params.J1, params.J2
    
    # xic = current_state x, xg = goal_state
    # Create reference trajectory for MPC horizon
    Xref, Uref = reference_trajectory(model, xic, xg, dt, N)
    
    # Update params with new references and initial condition
    mpc_params = (
        model = model,
        nx = params.nx,
        nu = params.nu, 
        dt = dt,
        N = N,
        M1 = M1,
        M2 = M2,
        J1 = J1,
        J2 = J2,
        xic = xic,
        xg = xg,
        idx = idx,
        Q = params.Q, 
        R = params.R, 
        Qf = params.Qf,
        Xref = Xref,
        Uref = Uref
    )
    
    #Keep the y position of all 3 bodies above ground (primal bound)
    x_l = -Inf*ones(idx.nz)
    x_u = Inf*ones(idx.nz)
    for i = 1:N
        x_l[idx.x[i]][2] = 0 #Body y position
        x_l[idx.x[i]][4] = 0 #Foot 1 y position
        x_l[idx.x[i]][6] = 0 #Foot 2 y position
    end
    
    # Inequality constraint bounds
    # length constraints between body and foot 1 / body and foot 2
    # bound between 0.5 and 1.5
    c_l = 0.5 * ones(2*N) # Minimum leg length
    c_u = 1.5 * ones(2*N) # Maximum leg length
    
    # Initialize z0 with reference trajectory Xref, Uref
    z0 = zeros(idx.nz) # update this
    for i = 1:(N-1)
        z0[idx.x[i]] .= Xref[i]
        z0[idx.u[i]] .= Uref[i]
    end
    z0[idx.x[N]] .= Xref[N]
    
    # adding a little noise to the initial guess is a good idea 
    z0 = z0 + (1e-6) * randn(idx.nz)
    
    diff_type = :auto
    
    Z = fmincon(walker_cost, walker_equality_constraint, walker_inequality_constraint,
                x_l, x_u, c_l, c_u, z0, mpc_params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 500, verbose = false)
    
    # get optimal control values
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    
    # Return first control input
    return U[1]
end

solve_mpc (generic function with 1 method)

In [6]:
# 10 hz
# but can run at 1/10 real time speed
# resolve trajopt problem often, close the loop in a sim
#make a simulator for the dynamics where you implement the dynamics in a normal hybrid dynamics rollout loop

#solve ipopt problem
#take initial conditioins and roll it out in simulator. and if there's no noise it should just simulate
#1. take dynamics from hw, code up as a simulator. refer to hybrid systems lecture. literally just roll out the dynamics
#compare the u's from the original problem and the simulator - should be same

#step 2. Add disturbance, it should totally not work
# take mpc problem

#add noise to controls not rk4
#xk+1 from simulation becomes xic in controller
# then loop back = this is closing the loop 


function run_mpc_simulation()
    # dynamics parameters
    model = (g = 9.81, mb = 5.0, mf = 1.0, ℓ_min = 0.5, ℓ_max = 1.5)
    
    # problem size
    nx = 12
    nu = 3
    tf = 4.4
    dt = 0.1 #10 Hz control rate
    t_vec = 0:dt:tf
    Nt = length(t_vec)
    
    # MPC horizon 
    mpc_N = 10 
    
    # initial and goal states
    xic = [-1.5;1;-1.5;0;-1.5;0;0;0;0;0;0;0]
    xg = [1.5;1;1.5;0;1.5;0;0;0;0;0;0;0]
    
    # index sets
    M1 = vcat([ (i-1)*10      .+ (1:5)   for i = 1:5]...)
    M2 = vcat([((i-1)*10 + 5) .+ (1:5)   for i = 1:4]...)
    J1 = [5,15,25,35]
    J2 = [10,20,30,40]
    
    # create indexing utilities
    idx = create_idx(nx, nu, mpc_N)
    
    # LQR cost function (tracking Xref, Uref) weights
    Q = diagm([1; 10; fill(1.0, 4); 1; 10; fill(1.0, 4)])
    R = diagm(fill(1e-3, 3))
    Qf = 1*Q
    
    # put everything useful in params
    mpc_params = (
        model = model,
        nx = nx,
        nu = nu,
        dt = dt,
        N = mpc_N,
        M1 = M1,
        M2 = M2,
        J1 = J1,
        J2 = J2,
        xic = xic,
        xg = xg,
        idx = idx,
        Q = Q,
        R = R,
        Qf = Qf
    )
    
    thist = t_vec
    xhist = zeros(nx, Nt)
    xhist[:,1] = xic
    uhist = []
    
    # CONTROL NOISE
    noise_level = 0.05  # 5% noise
    
    # simulation loop
    for k = 1:(Nt-1)

        x_current = xhist[:, k]
        
        # Stance constraints - basically guard function?
        if k in M1
            # In stance 1, pin foot 1 on ground
            x_current[4] = 0.0
        elseif k in M2
            # In stance 2, pin foot 2 on ground
            x_current[6] = 0.0
        end
        
        # solve MPC to get control input
        u_nominal = solve_mpc(x_current, xg, mpc_params)
        
        # add noise to input
        u_noise = noise_level * norm(u_nominal) * randn(3)
        u_current = u_nominal + u_noise
        push!(uhist, u_current)
        
        if k in M1 && !(k in J1)
            # stance 1 discrete dynamics without jump
            x_next = rk4(model, stance1_dynamics, x_current, u_current, dt)
        elseif k in M2 && !(k in J2)
            # stance 2 discrete dynamics without jump
            x_next = rk4(model, stance2_dynamics, x_current, u_current, dt)
        elseif k in J1
            # if at a mode switch from stance1 to stance2
            # integrate using stance1 dynamics then apply jump2_map
            x_next = rk4(model, stance1_dynamics, x_current, u_current, dt)
            println("Mode switch at step $k (stance1->stance2), applying jump2_map")
            x_next = jump2_map(x_next)
        elseif k in J2
            # if at a mode switch from stance2 to stance1
            # integrate using stance2 dynamics then apply jump1_map
            x_next = rk4(model, stance2_dynamics, x_current, u_current, dt)
            println("Mode switch at step $k (stance2->stance1), applying jump1_map")
            x_next = jump1_map(x_next)
        else
            x_next = x_current
        end
        
        xhist[:, k+1] = x_next
    end
    
    plot_results(thist, xhist, uhist)
    
    return thist, xhist, uhist
end


function plot_results(thist, xhist, uhist)
    Nt = length(thist)
    
    # Plot body & feet positions
    body_x = [xhist[1, k] for k in 1:Nt]
    body_y = [xhist[2, k] for k in 1:Nt]
    foot1_x = [xhist[3, k] for k in 1:Nt]
    foot1_y = [xhist[4, k] for k in 1:Nt]
    foot2_x = [xhist[5, k] for k in 1:Nt]
    foot2_y = [xhist[6, k] for k in 1:Nt]
    
    plt_positions = plot(body_x, body_y, label="Body", lw=2)
    plot!(plt_positions, foot1_x, foot1_y, label="Foot 1", lw=2)
    plot!(plt_positions, foot2_x, foot2_y, label="Foot 2", lw=2)
    xlabel!("x (m)")
    ylabel!("y (m)")
    title!("MPC Simulation with Noise")
    display(plt_positions)
    
    # plot control inputs over time:
    if length(uhist) > 0
        u1 = [u[1] for u in uhist]
        u2 = [u[2] for u in uhist]
        u3 = [u[3] for u in uhist]
        
        plt_controls = plot(thist[1:end-1], u1, label="F1", lw=2)
        plot!(plt_controls, thist[1:end-1], u2, label="F2", lw=2)
        plot!(plt_controls, thist[1:end-1], u3, label="τ", lw=2)
        xlabel!("Time (s)")
        ylabel!("Control inputs (U)")
        title!("MPC Closed Loop Controls with Noise")
        display(plt_controls)
    end
end

# Run!!!! GO!!!!
thist, xhist, uhist = run_mpc_simulation()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

---------IPOPT FAILED TO SOLVE----------------------
solver status: ITERATION_LIMIT


LoadError: ---------IPOPT FAILED TO SOLVE----------------------